In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
dbutils.fs.ls("/FileStore/tables")

Out[8]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1702493373798),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1702493373798),
 FileInfo(path='dbfs:/FileStore/tables/single_json_file.json', name='single_json_file.json', size=293, modificationTime=1701857903000),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1702493373798)]

In [0]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0e0816526d11-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0e0816526d11-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[11]: True

In [0]:
# list the topics stored on the mounted S3 bucket
display(dbutils.fs.ls("/mnt/user-0e0816526d11-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.geo/,0e0816526d11.geo/,0,1702493450799
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.pin/,0e0816526d11.pin/,0,1702493450799
dbfs:/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.user/,0e0816526d11.user/,0,1702493450799


In [0]:
# create path to topic files
file_location = "/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.pin/partition=0/*.json"
# specify file type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# load JSONs from mounted S3 bucket to Spark dataframe
df_pin = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

# cleaning Pinterest dataframe
df_pin = df_pin.replace("No description available Story format", None)
df_pin = df_pin.replace("null", None)
df_pin = df_pin.replace("User Info Error", None)
df_pin = df_pin.replace("Image src error", None)
df_pin = df_pin.replace("N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e", None)
df_pin = df_pin.replace("No Title Data Available", None)

# transforms the 'follower_count' col from string to integer and checks if the value matches a pattern that contains either k or M 
df_pin = df_pin.withColumn("follower_count", when(
    col("follower_count").rlike("\d+k"),(regexp_extract(col("follower_count"),"(\d+)",1).cast("integer") * 1000)).when(col("follower_count").rlike("\d+M"),(regexp_extract(col("follower_count"), "(\d+)", 1).cast("integer") * 1000000))
# otherwise, if it doesn't matches it leaves the full integer value
.otherwise(col("follower_count").cast("integer")))

# cleaning the 'save_location' column by removing 'Local save in ' text and just leaving the path for the 'save_location' column
df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# Renaming the column
df_pin = df_pin.withColumnRenamed("index", "ind")

# rearranging the Pinterest columns
reorder_col = ["ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category"]
df_pin = df_pin.select(reorder_col)

# Show the table of the Pinterest data
df_pin.show()

+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
4315|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| 25000| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| /data/education| education|
4315|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| 25000| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| /data/education| education|
5069|b75b6f87-deb3-444...|The Vault: Curate...|Sacramento Califo...| 6000000| Style Me Pretty|60th Anniversary ...| image|https://i.pinimg....|/data/event-planning|event-planning|
6063|60693727-4927-4bd...|41 Gorgeous Fall ...|Beautiful and eas...| 92000|Kristen | Lifesty...|Fall Home Decor,A...| image|https://i.pinimg....| /data/home-decor| home-decor|
3454|46bd3f86-b09d-4e2...|What can you use ...|HELPFUL RESOURCES...| 6000| Mixed Media Crafts|Epoxy Resin Art,D...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
5730|1e1f0c8b-9fcf-460...|Island Oasis Coup...|Description Coupo...| 0| Consuelo Aguirre|Grocery Items,Gro...| image|https://i.pinimg....| /data/finance| finance|
5730|1e1f0c8b-9fcf-460...|Island Oasis Coup...|Description Coupo...| 0| Consuelo Aguirre|Grocery Items,Gro...| image|https://i.pinimg....| /data/finance| finance|
5730|1e1f0c8b-9fcf-460...|Island Oasis Coup...|Description Coupo...| 0| Consuelo Aguirre|Grocery Items,Gro...| image|https://i.pinimg....| /data/finance| finance|
2863|9bf39437-42a6-4f0...|25 Super Fun Summ...|Keep the kids bus...| 124000|Of Life & Lisa | ...|Summer Crafts For...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
2863|9bf39437-42a6-4f0...|25 Super Fun Summ...|Keep the kids bus...| 124000|Of Life & Lisa | ...|Summer Crafts For...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
2863|9bf39437-42a6-4f0...|25 Super Fun Summ...|Keep the kids bus...| 124000|Of Life & Lisa | ...|Summer Crafts For...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
2923|52fa3af5-24a4-4cc...|UFO Paper Plate C...|A fun space activ...| 192000| The Crafting Chicks|Paper Plate Craft...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
8731|ea760f71-febf-402...|20 Koi Fish Tatto...|Koi fish tattoos ...| 211000| TheTrendSpotter|Dr Tattoo,Wörter ...| image|https://i.pinimg....| /data/tattoos| tattoos|
8731|ea760f71-febf-402...|20 Koi Fish Tatto...|Koi fish tattoos ...| 211000| TheTrendSpotter|Dr Tattoo,Wörter ...| image|https://i.pinimg....| /data/tattoos| tattoos|
8731|ea760f71-febf-402...|20 Koi Fish Tatto...|Koi fish tattoos ...| 211000| TheTrendSpotter|Dr Tattoo,Wörter ...| image|https://i.pinimg....| /data/tattoos| tattoos|
8304|5b6d0913-25e4-43a...|The #1 Reason You...|#lovequotes #matc...| 51000|Commitment Connec...|Wise Quotes,Quota...| image|https://i.pinimg....| /data/quotes| quotes|
8304|5b6d0913-25e4-43a...|The #1 Reason You...|#lovequotes #matc...| 51000|Commitment Connec...|Wise Quotes,Quota...| image|https://i.pinimg....| /data/quotes| quotes|
8304|5b6d0913-25e4-43a...|The #1 Reason You...|#lovequotes #matc...| 51000|Commitment Connec...|Wise Quotes,Quota...| image|https://i.pinimg....| /data/quotes| quotes|
+----+--------------------+--

In [0]:
file_location = "/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.geo/partition=0/*.json"
# specify file type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# load JSONs from mounted S3 bucket to Spark dataframe
df_geo = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

# created a new column containing latitude and longitude
df_geo = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

# dropping columns
df_geo = df_geo.drop("latitude", "longitude")

df_geo = df_geo.withColumn("timestamp", to_timestamp(df_geo["timestamp"], "yyyy-MM-dd HH:mm:ss"))

geo_reorder_col = ["ind", "country", "coordinates", "timestamp"]
df_geo = df_geo.select(geo_reorder_col)

# Show the table of the Geolocation data
df_geo.show()

+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 null| ["country"]|[["latitude"], ["...| null|
 null| ["country"]|[["latitude"], ["...| null|
 null| ["country"]|[["latitude"], ["...| null|
 null| ["country"]|[["latitude"], ["...| null|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
 6145| Mozambique|[-65.9079, -143.845]|2019-12-05 02:09:44|
 5494| Bulgaria|[-82.6768, -129.202]|2021-07-21 02:02:35|
 6063| Anguilla|[-89.1797, -174.015]|2021-07-20 09:02:47|
 3454| Cambodia|[-0.375174, 49.8106]|2021-07-25 02:20:29|
10625| Jamaica|[-84.4944, -81.0613]|2018-07-13 11:51:15|
 7528| Albania|[-89.9787, -173.293]|2020-08-28 03:52:47|
 2863| Armenia|[-5.34445, -177.924]|2020-04-27 13:34:16|
 5730| Colombia| [-77.015, -101.437]|2021-04-19 17:37:03|
 3089| Albania|[-89.9787, -173.293]|2018-02-28 05:31:29|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

In [0]:
file_location = "/mnt/user-0e0816526d11-bucket/topics/0e0816526d11.user/partition=0/*.json"
# specify file type
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# load JSONs from mounted S3 bucket to Spark dataframe
df_user = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)

df_user = df_user.withColumn("user_name", concat(col("first_name"),lit(" "),col("last_name")))
df_user = df_user.drop("first_name","last_name","index")
df_user = df_user.withColumn("date_joined", to_timestamp(df_user["date_joined"], "yyyy-MM-dd HH:mm:ss"))

user_reorder_col = ["ind","user_name","age","date_joined"]
df_user = df_user.select(user_reorder_col)

# Show the table of the User data
df_user.show()

+-----+--------------------+-------+-------------------+
 ind| user_name| age| date_joined|
+-----+--------------------+-------+-------------------+
 null|["first_name"] ["...|["age"]| null|
 null|["first_name"] ["...|["age"]| null|
 null|["first_name"] ["...|["age"]| null|
 null|["first_name"] ["...|["age"]| null|
 8731| Andrea Alexander| 21|2015-11-10 09:27:42|
 8731| Andrea Alexander| 21|2015-11-10 09:27:42|
 4315| Michelle Prince| 36|2015-12-20 16:38:13|
10625| Christian Lang| 32|2017-10-10 20:09:33|
 1313| Brittany Jones| 32|2016-04-02 03:51:23|
10794| Thomas Turner| 34|2016-12-22 00:02:02|
 1313| Brittany Jones| 32|2016-04-02 03:51:23|
 8304| Charles Berry| 25|2015-12-28 04:21:39|
 6063| Corey Andrews| 23|2015-11-25 13:36:22|
 3454| Robert Murphy| 48|2017-09-26 16:31:56|
 7554| Cheryl Huerta| 20|2017-04-11 16:35:33|
 6145| Emily Hawkins| 30|2015-12-25 20:24:37|
 8304| Charles Berry| 25|2015-12-28 04:21:39|
 8304| Charles Berry| 25|2015-12-28 04:21:39|
 2863| Dylan Holmes| 32|2016-10-23 14:06:51|
 5730| Rachel Davis| 36|2015-12-08 20:02:43|
+-----+--------------------+-------+-------------------+
only showing top 20 rows

In [0]:
dbutils.fs.unmount("/mnt/user-0e0816526d11-bucket")

/mnt/user-0e0816526d11-bucket has been unmounted.
Out[6]: True